# load data

In [50]:
import sqlite3
import pandas as pd
import re
from urllib.parse import urlparse
import requests

In [51]:
from datetime import date
from datetime import timedelta

In [52]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
    
from pipeline_functions import load_tweets

## load tweets older then two weeks

In [53]:
df_tweets = load_tweets("../home.db", days=21)
df_tweets = df_tweets[df_tweets.retweeted_status == "N/A"] # remove RT

# utils

In [54]:
df_tweets.tail()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
52118,1350953644070490121,198579020,New video!\n\n10 things that made me a senior ...,2021-01-17T23:50:22+00:00,en,N/A,N/A,0,N/A
52119,1350954099190222852,1480749782,This was the coup? \n https://twitter.com/alle...,2021-01-17T23:52:11+00:00,en,N/A,N/A,0,N/A
52120,1350954168794669057,2772007669,Who do you want to hear from this Waitangi Day?,2021-01-17T23:52:28+00:00,en,N/A,N/A,0,N/A
52122,1350954318103539713,1205226529455632385,. @AmyKremer is president of @WomenforTrump — ...,2021-01-17T23:53:03+00:00,en,N/A,N/A,0,N/A
52124,1350955020913684483,817386,"Surging demand for laptops and 5G chips, along...",2021-01-17T23:55:51+00:00,en,N/A,N/A,0,N/A


In [55]:
def find_url(tweet):
    return re.findall(r"http\S+", tweet)

def remove_tw_urls(tweet: str) -> str:
    tweet = re.sub(r"https://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://twitter.com/\S+", "", tweet)
    return tweet

def rem_short_links(tweet: str) -> str:
    tweet = re.sub(r"https://bit.ly/\S+", "", tweet)
    tweet = re.sub(r"http://bit.ly/\S+", "", tweet)
    tweet = re.sub(r"https://buff.ly/\S+", "", tweet)
    tweet = re.sub(r"http://buff.ly/\S+", "", tweet)
    tweet = re.sub(r"https://t.co/\S+", "", tweet)
    tweet = re.sub(r"http://t.co/\S+", "", tweet)

    return tweet

def expand_link(url):
    if url in ["https://", "http://"]:
        return url
    else:
        try:
            session = requests.Session()
            resp = session.head(url, allow_redirects=True)
            long_url = resp.url
            return long_url
        except:
            return url

def get_domain(url):
    domain = urlparse(url).netloc
    dot_split = domain.split(".")
#     if len(dot_split) > 2:
#         return ".".join(dot_split[1:])
    return domain


def remove_empty_str(l):
    for i in l:
        if len(i) == 0:
            l.remove(i)
    return l

# searching for not-expanded news urls

In [56]:
dfz = df_tweets[df_tweets.id == 1343991976534962179]
dfz["clean_text"] = dfz["full_text"].apply(remove_tw_urls)
dfz["clean_text"] = dfz["full_text"].apply(rem_short_links)
dfz["urls"] = dfz["clean_text"].apply(find_url)
dfz["domains"] = dfz.urls.apply(lambda x: [get_domain(d) for d in x])
dfz

<ipython-input-56-3a05ae6a8dd2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfz["clean_text"] = dfz["full_text"].apply(remove_tw_urls)
<ipython-input-56-3a05ae6a8dd2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfz["clean_text"] = dfz["full_text"].apply(rem_short_links)
<ipython-input-56-3a05ae6a8dd2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,clean_text,urls,domains
31378,1343991976534962179,794010396,"The UK Government, SAGE and their propaganda a...",2020-12-29T18:47:12+00:00,en,N/A,N/A,0,N/A,"The UK Government, SAGE and their propaganda a...",[https://www.bbc.co.uk/news/uk-55475240],[www.bbc.co.uk]


In [37]:
def remove_tw_urls(tweet):
    tweet = re.sub(r"https://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://twitter.com/\S+", "", tweet)
    return tweet

In [58]:
# dfz = df_tweets.copy()
# dfz["clean_text"] = dfz["full_text"].apply(remove_tw_urls)
# dfz["clean_text"] = dfz["clean_text"].apply(rem_short_links)
# dfz["urls"] = dfz["clean_text"].apply(find_url)
# dfz["domains"] = dfz.urls.apply(lambda x: [get_domain(d) for d in x])

<ipython-input-58-581e0d949cdd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfz["clean_text"] = dfz["full_text"].apply(remove_tw_urls)
<ipython-input-58-581e0d949cdd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfz["clean_text"] = dfz["clean_text"].apply(rem_short_links)
<ipython-input-58-581e0d949cdd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [59]:
dfz

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,clean_text,urls,domains
31378,1343991976534962179,794010396,"The UK Government, SAGE and their propaganda a...",2020-12-29T18:47:12+00:00,en,N/A,N/A,0,N/A,"The UK Government, SAGE and their propaganda a...",[https://www.bbc.co.uk/news/uk-55475240],[www.bbc.co.uk]


In [39]:
# dfz["urls_exp"] = dfz.urls.apply(lambda x: [expand_link(d) for d in x])

In [40]:
# dfz["domains_exp"] = dfz.urls_exp.apply(lambda x: [get_domain(d) for d in x])

In [60]:
dfz.head(20)

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,clean_text,urls,domains
31378,1343991976534962179,794010396,"The UK Government, SAGE and their propaganda a...",2020-12-29T18:47:12+00:00,en,N/A,N/A,0,N/A,"The UK Government, SAGE and their propaganda a...",[https://www.bbc.co.uk/news/uk-55475240],[www.bbc.co.uk]


In [61]:
new_columns_list = []
max_nr_dom = dfz.domains.str.len().max()
for i in range(max_nr_dom):
    new_columns_list.append(f"domain{i+1}")
dfz[new_columns_list] = pd.DataFrame(dfz.domains.tolist())

/home/gsajko/miniconda3/envs/mlb/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [62]:
dfz[["id", "urls", "domains"]][dfz.urls.apply(len) > 1]

,id,urls,domains


In [57]:
dfz

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,clean_text,urls,domains
31378,1343991976534962179,794010396,"The UK Government, SAGE and their propaganda a...",2020-12-29T18:47:12+00:00,en,N/A,N/A,0,N/A,"The UK Government, SAGE and their propaganda a...",[https://www.bbc.co.uk/news/uk-55475240],[www.bbc.co.uk]


In [63]:
dfz["domain1"]

31378    NaN
Name: domain1, dtype: object

In [64]:
domain_list = []
for i in dfz["domains"].tolist():
    for d in i:
        if d not in domain_list:
            domain_list.append(d)

In [65]:
domain_list

['www.bbc.co.uk']

In [46]:
# weird_url = ['api.',
# 't…',
# 't.co…',
# 't.c…',
# '…',]
# dfz[dfz["domain1"].isin(weird_url)]

# function for df

In [73]:
def find_news(df, news_domains_list):

    df["clean_text"] = df["full_text"].apply(
        remove_tw_urls
    )  # TODO can I chain .apply?
    df["clean_text"] = df["clean_text"].apply(rem_short_links)
    df["urls"] = df["clean_text"].apply(find_url)
    df.drop(["clean_text"], axis=1, inplace=True)
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    df.drop(["urls"], axis=1, inplace=True)

    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

    df.drop(["domains"], axis=1, inplace=True)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

def drop_contains(df, column_name, word_list):
    for string in word_list:
        df["lower"] = df["full_text"].str.lower()
        df = df[df["lower"].str.contains(string)]
        df.drop(["lower"], axis=1, inplace=True)
    return df

In [74]:
with open("news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())

In [75]:
%%time
dfz = find_news(dfz, news_domains)

CPU times: user 13.1 ms, sys: 29 µs, total: 13.1 ms
Wall time: 12.1 ms


<ipython-input-73-e8127230c71b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["clean_text"] = df["full_text"].apply(
<ipython-input-73-e8127230c71b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["clean_text"] = df["clean_text"].apply(rem_short_links)
<ipython-input-73-e8127230c71b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [76]:
dfz

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,contains_news
31378,1343991976534962179,794010396,"The UK Government, SAGE and their propaganda a...",2020-12-29T18:47:12+00:00,en,N/A,N/A,0,N/A,0


In [ ]:
%%time
df_tweets = find_news(df_tweets, news_domains)

In [ ]:
def news_in_qt_rt(df):
    #TODO refractor this
    df["all_news"] = df["contains_news"].copy()

    df_qt = df[["id", "contains_news"]].copy()
    df_qt.columns = ["quoted_status", "qt_news"]
    df = df.merge(df_qt, on="quoted_status", how="left")
    df["qt_news"] = df["qt_news"].fillna(0).astype(np.int64)
    df["all_news"] = df["qt_news"].astype(np.int64) + df["contains_news"].astype(np.int64)

    df_qt = df[["id", "contains_news"]].copy()
    df_qt.columns = ["in_reply_to_status_id", "rt_news"]
    df = df.merge(df_qt, on="in_reply_to_status_id", how="left")
    df["rt_news"] = df["rt_news"].fillna(0).astype(np.int64)
    df["all_news"] = df["rt_news"].astype(np.int64) + df["all_news"].astype(np.int64)
    
    return df


In [ ]:
df = df_tweets.copy()

In [ ]:
df.head()

In [ ]:
for col in ["in_reply_to_status_id"]:
    df[col] = df[col].fillna(0).astype(np.int64)

df = news_in_qt_rt(df)

In [ ]:
df_tweets[df_tweets.id == 1335980504735072256]

In [ ]:
df[df.id == 1335967310843060230]

In [ ]:
# I don't have QT in database

In [ ]:
1335980504735072256

In [ ]:
df.info()

In [ ]:
df["in_reply_to_status_id"] = df["in_reply_to_status_id"].fillna(0).astype(np.int64)

df_qt = df[["id", "contains_news"]].copy()
df_qt.columns = ["in_reply_to_status_id", "rt_news"]
df = df.merge(df_qt, on="in_reply_to_status_id", how="left")
df["rt_news"] = df["rt_news"].fillna(0).astype(np.int64)
df["all_news"] = df["rt_news"].astype(np.int64) + df["all_news"].astype(np.int64)

In [ ]:
df = df.merge(df_qt, on="in_reply_to_status_id", how="left")

In [ ]:
%%time
df_tweets = news_in_qt_rt(df_tweets)

In [ ]:
print(df_tweets.shape)

In [ ]:
df_tweets[df_tweets.id == 1197689942396174336]

# remove seen

In [ ]:
seen_tweets = pd.read_csv("seen.csv")

In [ ]:
seen_tweets.head()

In [ ]:
seen_tweets.info()

In [ ]:
seen_tweets.drop_duplicates(inplace=True)

In [ ]:
df_tweets[df_tweets["id"].isin(seen_tweets["tweet_id"].tolist())].shape

In [ ]:
%%time
print(df_tweets.shape)
df_tweets = df_tweets[~df_tweets["id"].isin(seen_tweets["tweet_id"].tolist())]

In [ ]:
# fiter out non english tweets

In [ ]:
df_tweets.shape

In [ ]:
df_tweets = df_tweets[df_tweets["lang"] == "en"]
df_tweets.shape

# suffle them and filter out news

In [ ]:
df_tweets[df_tweets["contains_news"] == 0].shape

In [ ]:
def drop_contains(df, column_name, str_list, lower=True):
    for string in str_list:
        if lower:
            df["filter"] = df[column_name].str.lower().copy()
        if not lower:
            df["filter"] = df[column_name].copy()
        df = df_tweets[~df["filter"].str.contains(string)]
        df = df.drop(["filter"], axis=1).copy()
    return df

In [ ]:
df_tweets.shape

In [ ]:
🍿

In [ ]:
def show_contains(df, column_name, str_list, lower=True):
    for string in str_list:
        if lower:
            df["filter"] = df[column_name].str.lower().copy()
        if not lower:
            df["filter"] = df[column_name].copy()
        df = df_tweets[df["filter"].str.contains(string)]
        df = df.drop(["filter"], axis=1).copy()
    return df

In [ ]:
str_to_show = ["🍿"]
show_contains(df_tweets, column_name="full_text", str_list = str_to_show, lower=False)

In [ ]:
str_to_drop = ["breaking:"]
df_tweets = drop_contains(df_tweets, column_name="full_text", str_list = str_to_drop)
df_tweets.shape

In [ ]:
str_to_drop = ["GOP"]
df_tweets = drop_contains(df_tweets, column_name="full_text", str_list = str_to_drop, lower=False)
df_tweets.shape

In [ ]:
string = "breaking:"

In [ ]:
df_tweets["lower"] = df_tweets["full_text"].str.lower().copy()
df_tweets = df_tweets[~df_tweets["lower"].str.contains(string)]
df_tweets.drop(["lower"], axis=1, inplace=True)

In [ ]:
df_tweets

In [ ]:
df_tweets["full_text"].str.lower()

In [ ]:
to_custom_news_feed = (
    df_tweets[df_tweets["contains_news"] == 0]
    .sample(frac=1)
    .reset_index(drop=True)[:1000]
)
to_custom_news_feed.shape

In [ ]:
to_custom_news_feed

In [ ]:
# to_custom_news_feed[["id", "user"]].to_csv("batch_to_add.csv")